In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [5]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Optional, Tuple, Union
import warnings
warnings.filterwarnings('ignore')
from src.config import RAW_DATA_DIR, PROCESSED_DATA_DIR, TRANSFORMED_DATA_DIR

# Set up directories
RAW_DIR = RAW_DATA_DIR
PROCESSED_DIR = PROCESSED_DATA_DIR
TRANSFORMED_DIR = TRANSFORMED_DATA_DIR


In [3]:
def load_station_mapping() -> pd.DataFrame:
    """Load station data and create rounded coordinates for weather mapping"""
    STATIONS_FILE = RAW_DIR / "-External-_Bluebikes_Station_List.xlsx"
    stations = pd.read_excel(STATIONS_FILE, header=1)
    
    # Create rounded coordinates for weather mapping
    stations["lat_round"] = stations["Lat"].round(1)
    stations["lon_round"] = stations["Long"].round(1)
    
    # Clean column names and select relevant columns
    stations = stations.rename(columns={
        "Station ID (to match to historic system data)": "station_id",
        "Number": "station_code",
        "NAME": "station_name",
        "Municipality": "municipality"
    })
    
    return stations[["station_id", "station_code", "station_name", "municipality", 
                    "Lat", "Long", "lat_round", "lon_round"]]

# Load stations
stations = load_station_mapping()
print(f"Loaded {len(stations)} stations")
print(stations.head())

Loaded 572 stations
  station_id station_code                          station_name municipality  \
0        461       L32001    Railroad Lot and Minuteman Bikeway    Arlington   
1        462       L32002       Linwood St at Minuteman Bikeway    Arlington   
2        480       L32005  Thorndike Field at Minuteman Bikeway    Arlington   
3        464       L32003                Mass Ave at Grafton St    Arlington   
4        465       L32004                Broadway at Grafton St    Arlington   

         Lat       Long  lat_round  lon_round  
0  42.416065 -71.153366       42.4      -71.2  
1  42.409354 -71.149065       42.4      -71.1  
2  42.400168 -71.144570       42.4      -71.1  
3  42.407261 -71.143821       42.4      -71.1  
4  42.409942 -71.140093       42.4      -71.1  


In [4]:
pip install holidays

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Optional
import holidays

def fill_missing_rides_full_range(df, hour_col, location_col, rides_col):
    """
    Fills in missing rides for all hours in the range and all unique locations.

    Parameters:
    - df: DataFrame with columns [hour_col, location_col, rides_col]
    - hour_col: Name of the column containing hourly timestamps
    - location_col: Name of the column containing location IDs
    - rides_col: Name of the column containing ride counts

    Returns:
    - DataFrame with missing hours and locations filled in with 0 rides
    """
    # Ensure the hour column is in datetime format
    df[hour_col] = pd.to_datetime(df[hour_col])

    # Get the full range of hours (from min to max) with hourly frequency
    full_hours = pd.date_range(
        start=df[hour_col].min(),
        end=df[hour_col].max(),
        freq="h"
    )

    # Get all unique location IDs
    all_locations = df[location_col].unique()

    # Create a DataFrame with all combinations of hours and locations
    full_combinations = pd.DataFrame(
        [(hour, location) for hour in full_hours for location in all_locations],
        columns=[hour_col, location_col]
    )

    # Merge the original DataFrame with the full combinations DataFrame
    merged_df = pd.merge(full_combinations, df, on=[hour_col, location_col], how='left')

    # Fill missing rides with 0
    merged_df[rides_col] = merged_df[rides_col].fillna(0).astype(int)

    return merged_df

def is_us_holiday(date):
    """Check if a date is a US holiday"""
    us_holidays = holidays.UnitedStates()
    return date in us_holidays

def load_and_aggregate_rides(years: List[int]) -> pd.DataFrame:
    """Load ride data and aggregate to hourly counts per station with complete time series"""
    all_rides = []
    
    for year in years:
        # Determine months based on year
        if year == 2024:
            months = range(10, 13)  # october to december for 2024
        else:
            months = range(1, 10)  # through september for 2025
            
        for month in months:
            file_path = RAW_DIR / f"rides_{year}_{month:02}.parquet"
            if file_path.exists():
                print(f"Loading {file_path.name}")
                
                # Read rides data
                rides = pd.read_parquet(file_path)
                
                # Convert timestamps and filter valid rides
                rides["started_at"] = pd.to_datetime(rides["started_at"], errors="coerce")
                rides = rides.dropna(subset=["started_at", "start_station_id"])
                
                # Round to hour and aggregate
                rides["hour"] = rides["started_at"].dt.floor("H")
                hourly_rides = rides.groupby(["hour", "start_station_id"]).size().reset_index(name="ride_count")
                
                all_rides.append(hourly_rides)
    
    if not all_rides:
        raise ValueError("No ride data found!")
    
    # Combine all data
    combined_rides = pd.concat(all_rides, ignore_index=True)
    
    print(f"Before filling: {len(combined_rides):,} records")
    
    # Apply the fill_missing_rides_full_range function
    combined_rides = fill_missing_rides_full_range(
        combined_rides, 
        hour_col="hour", 
        location_col="start_station_id", 
        rides_col="ride_count"
    )
    
    print(f"After filling: {len(combined_rides):,} records")
    
    # Add ONLY the required time features
    #combined_rides["is_weekend"] = combined_rides["hour"].dt.dayofweek.isin([5, 6]).astype(int)
    
    # Add US holiday feature
    #combined_rides["is_us_holiday"] = combined_rides["hour"].dt.date.apply(is_us_holiday).astype(int)
    
    # Sort by station and hour for consistency
    combined_rides = combined_rides.sort_values(["start_station_id", "hour"]).reset_index(drop=True)
    
    return combined_rides


# Load rides data with complete time series filling
rides_df = load_and_aggregate_rides([2024, 2025])
print(f"Loaded {len(rides_df):,} hourly ride records")
print(f"Time range: {rides_df['hour'].min()} to {rides_df['hour'].max()}")
print(f"Stations: {rides_df['start_station_id'].nunique()}")
print(f"Columns: {list(rides_df.columns)}")

# Display sample data
print("\nSample of filled rides data:")
print(rides_df.head(10))

Loading rides_2024_10.parquet
Loading rides_2024_11.parquet
Loading rides_2024_12.parquet
Loading rides_2025_01.parquet
Loading rides_2025_02.parquet
Loading rides_2025_03.parquet
Loading rides_2025_04.parquet
Loading rides_2025_05.parquet
Loading rides_2025_06.parquet
Loading rides_2025_07.parquet
Loading rides_2025_08.parquet
Loading rides_2025_09.parquet
Before filling: 1,638,389 records
After filling: 5,187,319 records
Loaded 5,187,319 hourly ride records
Time range: 2024-09-30 22:00:00 to 2025-09-30 23:00:00
Stations: 592
Columns: ['hour', 'start_station_id', 'ride_count']

Sample of filled rides data:
                 hour start_station_id  ride_count
0 2024-09-30 22:00:00           A32000           0
1 2024-09-30 23:00:00           A32000           0
2 2024-10-01 00:00:00           A32000           0
3 2024-10-01 01:00:00           A32000           0
4 2024-10-01 02:00:00           A32000           0
5 2024-10-01 03:00:00           A32000           0
6 2024-10-01 04:00:00       

In [5]:
def load_and_process_weather(years: List[int]) -> pd.DataFrame:
    """Load and process weather data - keep all raw weather metrics without grading"""
    all_weather = []
    
    for year in years:
        if year == 2024:
            months = range(10, 13)
        else:
            months = range(1, 10)
            
        for month in months:
            file_path = RAW_DIR / f"weather_{year}_{month:02}.parquet"
            if file_path.exists():
                print(f"Loading {file_path.name}")
                weather = pd.read_parquet(file_path)
                
                # Ensure proper datetime and numeric types
                weather["timestamp"] = pd.to_datetime(weather["timestamp"], errors="coerce")
                weather = weather.dropna(subset=["timestamp"])
                
                # Round to hour and take mean if multiple records per hour
                weather["hour"] = weather["timestamp"].dt.floor("H")
                weather_agg = weather.groupby(["hour", "lat_round", "lon_round"]).agg({
                    "temp": "mean",
                    "humidity": "mean", 
                    "pressure": "mean",
                    "wind_speed": "mean",
                    "wind_deg": "mean",
                    "precipitation": "mean"
                }).reset_index()
                
                all_weather.append(weather_agg)
    
    if not all_weather:
        raise ValueError("No weather data found!")
    
    combined_weather = pd.concat(all_weather, ignore_index=True)
    
    # Remove weather grade creation - just return raw weather metrics
    print(f"Weather columns: {list(combined_weather.columns)}")
    
    return combined_weather

# Load weather data
weather_df = load_and_process_weather([2024, 2025])
print(f"Loaded {len(weather_df):,} hourly weather records")

Loading weather_2024_10.parquet
Loading weather_2024_11.parquet
Loading weather_2024_12.parquet
Loading weather_2025_01.parquet
Loading weather_2025_02.parquet
Loading weather_2025_03.parquet
Loading weather_2025_04.parquet
Loading weather_2025_05.parquet
Loading weather_2025_06.parquet
Loading weather_2025_07.parquet
Loading weather_2025_08.parquet
Loading weather_2025_09.parquet
Weather columns: ['hour', 'lat_round', 'lon_round', 'temp', 'humidity', 'pressure', 'wind_speed', 'wind_deg', 'precipitation']
Loaded 61,250 hourly weather records


In [35]:
def merge_rides_weather(rides_df: pd.DataFrame, weather_df: pd.DataFrame, stations: pd.DataFrame) -> pd.DataFrame:
    """Merge rides data with weather data using station coordinates"""
    
    # First, ensure both hour columns are timezone-naive for proper merging
    rides_df = rides_df.copy()
    weather_df = weather_df.copy()
    
    # Remove timezone from weather hour if it exists
    if rides_df['hour'].dt.tz is not None:
        rides_df['hour'] = rides_df['hour'].dt.tz_localize(None)
    if weather_df['hour'].dt.tz is not None:
        weather_df['hour'] = weather_df['hour'].dt.tz_localize(None)
    
    print(f"Rides hour timezone: {rides_df['hour'].dt.tz}")
    print(f"Weather hour timezone: {weather_df['hour'].dt.tz}")
    
    # Merge rides with station information to get coordinates
    rides_with_coords = rides_df.merge(
        stations[["station_code", "lat_round", "lon_round"]].rename(columns={"station_code": "start_station_id"}),
        left_on="start_station_id", 
        right_on="start_station_id",
        how="left"
    )
    
    print(f"Rides with coords: {len(rides_with_coords):,} records")
    print(f"Available weather coordinates: {weather_df[['lat_round', 'lon_round']].drop_duplicates().shape[0]} unique pairs")
    
    # Check for missing coordinates after merge
    missing_coords = rides_with_coords['lat_round'].isnull().sum()
    if missing_coords > 0:
        print(f"Warning: {missing_coords} records missing coordinates after station merge")
        rides_with_coords = rides_with_coords.dropna(subset=['lat_round', 'lon_round'])
    
    # Merge with weather data
    merged_df = rides_with_coords.merge(
        weather_df,
        left_on=["hour", "lat_round", "lon_round"],
        right_on=["hour", "lat_round", "lon_round"],
        how="left"
    )
    
    print(f"Merged dataset: {len(merged_df):,} records")
    
    # Define weather columns (remove weather_grade since we don't want it)
    weather_cols = ["temp", "humidity", "pressure", "wind_speed", "wind_deg", "precipitation"]
    
    # Check which weather columns actually exist in the merged data
    available_weather_cols = [col for col in weather_cols if col in merged_df.columns]
    print(f"Available weather columns: {available_weather_cols}")
    
    # Sort by station and time for proper filling
    merged_df = merged_df.sort_values(["start_station_id", "hour"])
    
    # Fill missing values by station group only for available weather columns
    for col in available_weather_cols:
        missing_before = merged_df[col].isnull().sum()
        merged_df[col] = merged_df.groupby("start_station_id")[col].transform(
            lambda x: x.fillna(method='ffill').fillna(method='bfill')
        )
        missing_after = merged_df[col].isnull().sum()
        print(f"  {col}: filled {missing_before - missing_after} missing values")
    
    # For any remaining missing values, use station-specific mean
    for col in available_weather_cols:
        remaining_missing = merged_df[col].isnull().sum()
        if remaining_missing > 0:
            merged_df[col] = merged_df.groupby("start_station_id")[col].transform(
                lambda x: x.fillna(x.mean())
            )
            final_missing = merged_df[col].isnull().sum()
            print(f"  {col}: filled remaining {remaining_missing - final_missing} with station mean")
    
    return merged_df

# Merge the datasets
merged_data = merge_rides_weather(rides_df, weather_df, stations)
print(f"Final merged dataset: {len(merged_data):,} records")
print(f"Columns: {list(merged_data.columns)}")

Rides hour timezone: None
Weather hour timezone: None
Rides with coords: 5,187,319 records
Available weather coordinates: 7 unique pairs
Merged dataset: 4,950,745 records
Available weather columns: ['temp', 'humidity', 'pressure', 'wind_speed', 'wind_deg', 'precipitation']
  temp: filled 0 missing values
  humidity: filled 0 missing values
  pressure: filled 0 missing values
  wind_speed: filled 0 missing values
  wind_deg: filled 0 missing values
  precipitation: filled 0 missing values
  temp: filled remaining 0 with station mean
  humidity: filled remaining 0 with station mean
  pressure: filled remaining 0 with station mean
  wind_speed: filled remaining 0 with station mean
  wind_deg: filled remaining 0 with station mean
  precipitation: filled remaining 0 with station mean
Final merged dataset: 4,950,745 records
Columns: ['hour', 'start_station_id', 'ride_count', 'lat_round', 'lon_round', 'temp', 'humidity', 'pressure', 'wind_speed', 'wind_deg', 'precipitation']
